In [1]:
import torch

print("Torch version :", torch.__version__)
print("CUDA dispo :", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU :", torch.cuda.get_device_name(0))

x = torch.rand(1000, 1000)
y = torch.mm(x, x)

print("CPU ok :", y.shape)

if torch.cuda.is_available():
    x = x.to("cuda")
    y = torch.mm(x, x)
    print("GPU ok :", y.shape)


Torch version : 2.5.1+cu121
CUDA dispo : True
GPU : NVIDIA GeForce RTX 3060 Laptop GPU
CPU ok : torch.Size([1000, 1000])
GPU ok : torch.Size([1000, 1000])


### Si tout marche bien coté installation/fonctionnement du matos on commence


# Bench_matmul (CPU vs GPU)
le TPU se fait sur collab

In [4]:
import torch
import time
import numpy as np
import csv
from pathlib import Path

# =========================
# Configuration générale
# =========================

SIZES = { #on def les tailles pour tester
    "S": 1024,
    "M": 4096,
    "L": 8192,  #changer selon votre RAM et GPU, comme j'ai mit sur discord, je suis à 16GB de RAM et j'ai une RTX 3060
}

WARMUP = 20 # nombre d'itérations de warmup pour "chauffer" le cache et les optimisations
ITERS = 100 
DTYPE = torch.float32

RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

CSV_PATH = RESULTS_DIR / "matmul_cpu_gpu.csv"


# =========================
# Outils statistiques
# =========================

def compute_stats(times_ms):
    times = np.array(times_ms)
    return {
        "mean_ms": times.mean(),
        "p50_ms": np.percentile(times, 50),
        "p95_ms": np.percentile(times, 95),
    }


def gflops(n, time_ms):
    flops = 2 * (n ** 3)
    return flops / (time_ms / 1000) / 1e9


# =========================
# Benchmark CPU
# =========================

def bench_cpu(n):
    x = torch.rand((n, n), dtype=DTYPE)
    y = torch.rand((n, n), dtype=DTYPE)

    # Warmup
    for _ in range(WARMUP):
        torch.mm(x, y)

    times = []

    for _ in range(ITERS):
        start = time.perf_counter()
        torch.mm(x, y)
        end = time.perf_counter()

        times.append((end - start) * 1000)

    stats = compute_stats(times)
    stats["gflops"] = gflops(n, stats["mean_ms"])
    stats["h2d_ms"] = 0
    stats["compute_ms"] = stats["mean_ms"]
    stats["d2h_ms"] = 0

    return stats


# =========================
# Benchmark GPU
# =========================

def bench_gpu(n):
    device = "cuda"

    x_cpu = torch.rand((n, n), dtype=DTYPE)
    y_cpu = torch.rand((n, n), dtype=DTYPE)

    # Warmup complet
    for _ in range(WARMUP):
        x = x_cpu.to(device)
        y = y_cpu.to(device)
        torch.cuda.synchronize()
        torch.mm(x, y)
        torch.cuda.synchronize()

    times = []
    h2d_list = []
    compute_list = []
    d2h_list = []

    for _ in range(ITERS):
        # H → D
        t0 = time.perf_counter()
        x = x_cpu.to(device)
        y = y_cpu.to(device)
        torch.cuda.synchronize()
        h2d = (time.perf_counter() - t0) * 1000

        # Compute (CUDA events)
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        out = torch.mm(x, y)
        end.record()

        torch.cuda.synchronize()
        compute = start.elapsed_time(end)

        # D → H
        t0 = time.perf_counter()
        out.cpu()
        torch.cuda.synchronize()
        d2h = (time.perf_counter() - t0) * 1000

        total = h2d + compute + d2h

        times.append(total)
        h2d_list.append(h2d)
        compute_list.append(compute)
        d2h_list.append(d2h)

    stats = compute_stats(times)
    stats["gflops"] = gflops(n, stats["mean_ms"])
    stats["h2d_ms"] = np.mean(h2d_list)
    stats["compute_ms"] = np.mean(compute_list)
    stats["d2h_ms"] = np.mean(d2h_list)

    return stats


# =========================
# Sauvegarde CSV
# =========================

def save_row(device, size_tag, n, stats):
    file_exists = CSV_PATH.exists()

    with open(CSV_PATH, "a", newline="") as f:
        writer = csv.writer(f)

        if not file_exists:
            writer.writerow([
                "device", "size", "N",
                "mean_ms", "p50_ms", "p95_ms",
                "gflops",
                "h2d_ms", "compute_ms", "d2h_ms"
            ])

        writer.writerow([
            device, size_tag, n,
            stats["mean_ms"], stats["p50_ms"], stats["p95_ms"],
            stats["gflops"],
            stats["h2d_ms"], stats["compute_ms"], stats["d2h_ms"]
        ])


# =========================
# Exécution principale
# =========================

def main():
    print("Début des tests de multiplication matricielle\n")
    print("\n Ici on va avoir : - la taille de la matrice (N) \n- le temps moyen d'exécution (mean_ms) \n- les percentiles 50 et 95 (p50_ms, p95_ms)(par exemple p50 représente le temps typique d'exécution (50% rapide, 50% lent) )"
          " \n- les GFLOPS calculés (gflops) \n- pour le GPU, on aura aussi les temps de transfert Host to Device (h2d_ms), de calcul (compute_ms) et Device to Host (d2h_ms)\n")

    for size_tag, n in SIZES.items():
        print(f"\n=== Taille {size_tag} (N={n}) ===")

        # CPU
        cpu_stats = bench_cpu(n)
        print("CPU:", cpu_stats)
        save_row("cpu", size_tag, n, cpu_stats)

        # GPU
        if torch.cuda.is_available():
            gpu_stats = bench_gpu(n)
            print("GPU:", gpu_stats)
            save_row("gpu", size_tag, n, gpu_stats)

    


if __name__ == "__main__":
    main()


Début des tests de multiplication matricielle

Ici on va avoir : - la taille de la matrice (N) 
- le temps moyen d'exécution (mean_ms) 
- les percentiles 50 et 95 (p50_ms, p95_ms)(par exemple p50 représente le temps typique d'exécution (50% rapide, 50% lent) ) 
- les GFLOPS calculés (gflops) 
- pour le GPU, on aura aussi les temps de transfert Host to Device (h2d_ms), de calcul (compute_ms) et Device to Host (d2h_ms)


=== Taille S (N=1024) ===
CPU: {'mean_ms': 4.463385000008202, 'p50_ms': 4.8543000002609915, 'p95_ms': 5.240304999369982, 'gflops': 481.1334106280444, 'h2d_ms': 0, 'compute_ms': 4.463385000008202, 'd2h_ms': 0}
GPU: {'mean_ms': 3.679195997071365, 'p50_ms': 2.396276004426909, 'p95_ms': 10.633936304532833, 'gflops': 583.6828616114484, 'h2d_ms': 1.5788440000414994, 'compute_ms': 1.21033599704504, 'd2h_ms': 0.8900159999848256}

=== Taille M (N=4096) ===
CPU: {'mean_ms': 276.1603730000297, 'p50_ms': 271.9075000004523, 'p95_ms': 328.8107949997084, 'gflops': 497.67804112860614, '

Les résultats du test MatMul montrent que le GPU RTX 3060 n’apporte qu’un gain limité pour les petites matrices (S) en raison du coût des transferts mémoire et de la sous-utilisation de celui-ci.


En revanche, pour des tailles moyennes et grandes, le GPU atteint un débit d’environ 3,2 TFLOPS, soit un facteur d’accélération proche de ×6 par rapport au CPU.
